In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

CNN 基本上是几层具有非线性激活函数的卷积，以及将池化层应用于卷积的结果。每层应用不同的滤波器（成百上千个）。理解的关键是滤波器不是预先设定好的，而是在训练阶段学习的，以使得恰当的损失函数被最小化。已经观察到，较低层会学习检测基本特征，而较高层检测更复杂的特征，例如形状或面部。

请注意，由于有池化层，靠后的层中的神经元看到的更多的是原始图像，因此它们能够编辑前几层中学习的基本特征。

到目前为止，描述了 ConvNet 的基本概念。CNN 在时间维度上对音频和文本数据进行一维卷积和池化操作，沿（高度×宽度）维度对图像进行二维处理，沿（高度×宽度×时间）维度对视频进行三维处理。对于图像，在输入上滑动滤波器会生成一个特征图，为每个空间位置提供滤波器的响应。

换句话说，一个 ConvNet 由多个滤波器堆叠在一起，学习识别在图像中独立于位置信息的具体视觉特征。这些视觉特征在网络的前面几层很简单，然后随着网络的加深，组合成更加复杂的全局特征。

## 局部感受野  

如果想保留图像中的空间信息，那么用像素矩阵表示每个图像是很方便的。然后，编码局部结构的简单方法是将相邻输入神经元的子矩阵连接成属于下一层的单隐藏层神经元。这个单隐藏层神经元代表一个局部感受野。请注意，此操作名为“卷积”，此类网络也因此而得名。  
当然，可以通过重叠的子矩阵来编码更多的信息。例如，假设每个子矩阵的大小是 5×5，并且将这些子矩阵应用到 28×28 像素的 MNIST 图像。然后，就能够在下一隐藏层中生成 23×23 的局部感受野。事实上，在触及图像的边界之前，只需要滑动子矩阵 23 个位置。  
定义从一层到另一层的特征图。当然，可以有多个独立从每个隐藏层学习的特征映射。例如，可以从 28×28 输入神经元开始处理 MNIST 图像，然后（还是以 5×5 的步幅）在下一个隐藏层中得到每个大小为 23×23 的神经元的 k 个特征图。

## 共享权重和偏置

假设想要从原始像素表示中获得移除与输入图像中位置信息无关的相同特征的能力。一个简单的直觉就是对隐藏层中的所有神经元使用相同的权重和偏置。通过这种方式，每层将从图像中学习到独立于位置信息的潜在特征。


## 池化层

假设我们要总结一个特征映射的输出。我们可以使用从单个特征映射产生的输出的空间邻接性，并将子矩阵的值聚合成单个输出值，从而合成地描述与该物理区域相关联的含义。
* 
### 最大池化  
一个简单而通用的选择是所谓的最大池化算子，它只是输出在区域中观察到的最大输入值。在 TensorFlow 中，如果想要定义一个大小为 2×2 的最大池化层，可以这样写：

In [ ]:
tf.nn.max_pool()

* 
### 平均池化
另一个选择是平均池化，它简单地将一个区域聚合成在该区域观察到的输入值的平均值。

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
#载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [5]:
x = tf.placeholder("float", shape=[None, 784])
y = tf.placeholder("float", shape=[None, 10])

In [6]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x,W) + b)

In [14]:
loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
gd = tf.train.AdagradOptimizer(0.2)
train = gd.minimize(loss_cross_entropy)

In [15]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
sess.run(tf.global_variables_initializer())

In [19]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    sess.run(train, feed_dict={x: batch[0], y: batch[1]})
print(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))

0.9182


In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [6]:
def conv2d(x, W):
   return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
   return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
with tf.name_scope('input'):
    x = tf.placeholder("float", shape=[None, 784])
    y = tf.placeholder("float", shape=[None, 10])

with tf.name_scope('inference'):
    x_image = tf.reshape(x, [-1,28,28,1])
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder("float")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    prediction_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


In [8]:
with tf.name_scope('loss'):
    loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction_conv))
    gd = tf.train.AdamOptimizer(1e-4)
    train = gd.minimize(loss_cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(prediction_conv, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
summary_writer = tf.summary.FileWriter('logs/', flush_secs=60)
summary_writer.add_graph(sess.graph)#添加graph图
tf.summary.scalar('loss', loss_cross_entropy)
tf.summary.scalar('accuracy', accuracy)
merged_summary_op = tf.summary.merge_all()#自动管理

In [11]:
sess.run(tf.global_variables_initializer())

In [ ]:
n_batch = 50
for epoch in range(2000): 
    for batch in range(n_batch): #每一個 outer loop 疊代 n_batch 個批次
        batch_xs, batch_ys = mnist.train.next_batch(50)
        feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 0.7} 
        sess.run(train, feed_dict)
    #計算一次準確率
    train_feed_dict = {x: mnist.train.images, y: mnist.train.labels, keep_prob: 1.0} #train data feed dictionary
    train_acc = sess.run(accuracy, train_feed_dict)
    test_feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0} #testing data feed dictionary
    test_acc = sess.run(accuracy, test_feed_dict)          
    print("Iter=" + str(epoch) + ", Training Accuracy=" + str(train_acc) + ", Testing Accuracy=" + str(test_acc))
    summary_str = sess.run(merged_summary_op, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
    summary_writer.add_summary(summary_str, global_step=step)
        

In [ ]:
for step in range(30000):
    batch = mnist.train.next_batch(50)
    sess.run(train, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.7})
    print(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))
#     train_feed_dict = {x: mnist.train.images, y: mnist.train.labels, keep_prob: 1.0} #train data feed dictionary
#     train_acc = sess.run(accuracy, train_feed_dict)
#     test_feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0} #testing data feed dictionary
#     test_acc = sess.run(accuracy, test_feed_dict)          
#     print("Iter=" + str(epoch) + ", Training Accuracy=" + str(train_acc) + ", Testing Accuracy=" + str(test_acc))
    summary_str = sess.run(merged_summary_op, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
    summary_writer.add_summary(summary_str, global_step=step)

0.1025
0.1196
0.1328
0.1368
0.1492
0.1625
0.1711
0.1724
0.1762
0.1807
0.1842
0.1851
0.1841
0.1825
0.1795
0.1806
0.183
0.1844
0.188
0.1856
0.1827
0.1822
0.1859
0.1882
0.1932
0.195
0.1958
0.1953
0.1962
0.1977
0.211
0.2419
0.2677
0.2864
0.2996
0.3056
0.3039
0.3025
0.307
0.3095
0.319
0.33
0.3376
0.3403
0.3462
0.3541
0.3626
0.3726
0.3896
0.4022
0.4143
0.4262
0.4353
0.445
0.4546
0.4655
0.4738
0.489
0.4977
0.5001
0.5026
0.5032
0.4999
0.5007
0.506
0.5105
0.5146
0.5189
0.5225
0.5256
0.5279
0.5277
0.527
0.5297
0.5312
0.531
0.5343
0.5366
0.5401
0.5409
0.5429
0.5446
0.5448
0.5448
0.5463
0.5478
0.5479
0.5474
0.5481
0.5498
0.5501
0.55
0.5511
0.5518
0.5518
0.5532
0.5534
0.5515
0.5512
0.5506
0.5531
0.5551
0.5551
0.555
0.5499
0.5433
0.541
0.5406
0.5427
0.5468
0.5494
0.5525
0.5574
0.5637
0.5705
0.569
0.5684
0.5667
0.5647
0.563
0.5619
0.5617
0.563
0.5628
0.5624
0.5615
0.5612
0.5617
0.5622
0.5628
0.5631
0.5632
0.5619
0.5595
0.5596
0.5592
0.5596
0.5601
0.5648
0.568
0.5778
0.5874
0.6027
0.6154
0.6174
0.6164

0.8673
0.8676
0.8674
0.8678
0.8678
0.8679
0.8681
0.8678
0.8682
0.8668
0.8664
0.8659
0.866
0.8663
0.8663
0.8672
0.8679
0.8687
0.8695
0.87
0.87
0.8705
0.8709
0.8712
0.8714
0.8714
0.8714
0.8717
0.8719
0.8717
0.8716
0.8714
0.8716
0.8716
0.8713
0.871
0.8707
0.8708
0.8707
0.8702
0.8701
0.8702
0.8701
0.8699
0.8702
0.8705
0.8706
0.8708
0.8713
0.8714
0.8719
0.8718
0.8716
0.8726
0.8721
0.8718
0.8717
0.8714
0.8717
0.8718
0.8723
0.8722
0.872
0.8717
0.8719
0.8724
0.8727
0.8717
0.8711
0.8707
0.869
0.868
0.8663
0.8652
0.8652
0.8654
0.8656
0.867
0.8675
0.8678
0.8688
0.869
0.8691
0.8696
0.8699
0.8699
0.87
0.8702
0.8703
0.8703
0.8702
0.8712
0.8713
0.8715
0.8707
0.8704
0.8708
0.8708
0.8708
0.8709
0.8713
0.8713
0.8712
0.8711
0.8715
0.8712
0.8709
0.8707
0.8703
0.8703
0.8704
0.8707
0.871
0.8706
0.8708
0.8717
0.8732
0.8758
0.8812
0.8888
0.8987
0.9149
0.9274
0.9257
0.9141
0.9017
0.8866
0.8854
0.9021
0.9206
0.9326
0.9358
0.9328
0.9303
0.9281
0.926
0.9264
0.9293
0.9282
0.9273
0.9266
0.9301
0.9319
0.9342
0.9408


0.9746
0.9744
0.9744
0.9749
0.9751
0.9752
0.9753
0.9749
0.9739
0.974
0.9736
0.9735
0.9734
0.9735
0.9737
0.9741
0.9747
0.9742
0.9746
0.9746
0.9746
0.9748
0.9748
0.975
0.9756
0.9753
0.9757
0.9763
0.9766
0.9769
0.9769
0.9774
0.9772
0.977
0.9768
0.9767
0.9765
0.9766
0.9768
0.9766
0.9763
0.9765
0.9768
0.9766
0.9769
0.9769
0.977
0.9772
0.9772
0.977
0.9765
0.9763
0.9763
0.9764
0.9766
0.9762
0.9765
0.9763
0.9764
0.9763
0.9764
0.9766
0.9765
0.9765
0.9761
0.9759
0.9766
0.9767
0.9766
0.9773
0.9769
0.9772
0.9771
0.9768
0.9768
0.9768
0.9767
0.9766
0.9771
0.9772
0.9775
0.9778
0.9774
0.9777
0.9773
0.9772
0.9777
0.9773
0.9773
0.9772
0.9774
0.9777
0.9774
0.9773
0.9776
0.9775
0.9775
0.9776
0.9775
0.977
0.9769
0.977
0.9771
0.9762
0.9754
0.975
0.9752
0.9752
0.9752
0.9749
0.9755
0.9757
0.9764
0.9764
0.9767
0.977
0.9771
0.977
0.9773
0.9771
0.9774
0.9772
0.9774
0.9775
0.9774
0.9773
0.9776
0.9772
0.9773
0.9776
0.9774
0.977
0.9774
0.9777
0.9782
0.9781
0.9773
0.9771
0.9772
0.9772
0.9771
0.9774
0.9771
0.9769
0.9

0.983
0.9828
0.9827
0.982
0.9824
0.9825
0.9825
0.9832
0.9821
0.9815
0.9812
0.9806
0.9804
0.9799
0.9796
0.9801
0.9797
0.9792
0.9786
0.9798
0.9805
0.9812
0.9813
0.9816
0.982
0.9825
0.983
0.9828
0.9832
0.9831
0.9827
0.983
0.9825
0.9826
0.9822
0.982
0.9823
0.9817
0.982
0.982
0.9818
0.9815
0.9816
0.9817
0.9819
0.9813
0.9813
0.9811
0.9813
0.9813
0.9814
0.9819
0.9816
0.9819
0.9821
0.9823
0.9826
0.983
0.9831
0.9832
0.9832
0.9834
0.9834
0.9835
0.984
0.9838
0.9837
0.9836
0.9838
0.9837
0.9835
0.9836
0.9837
0.9836
0.9843
0.9841
0.9843
0.9843
0.9844
0.9848
0.9847
0.9848
0.9845
0.9838
0.9837
0.9834
0.9833
0.9831
0.9827
0.9819
0.9822
0.9821
0.9818
0.9819
0.9815
0.9817
0.9813
0.9814
0.982
0.9817
0.9817
0.9809
0.9802
0.9799
0.9794
0.979
0.9795
0.9798
0.9802
0.9804
0.9811
0.9807
0.9809
0.9812
0.9811
0.9812
0.9818
0.9818
0.9808
0.98
0.9805
0.9806
0.9807
0.981
0.9813
0.9816
0.9816
0.982
0.9821
0.9825
0.9826
0.9825
0.9828
0.9828
0.9834
0.9831
0.9831
0.9834
0.9829
0.9826
0.9825
0.9825
0.9822
0.9825
0.9825
0

0.9825
0.9823
0.9823
0.9821
0.9821
0.9825
0.983
0.9835
0.9837
0.9838
0.9843
0.9842
0.9844
0.9842
0.9843
0.9837
0.9836
0.9835
0.9835
0.9838
0.9836
0.9836
0.9835
0.9831
0.9832
0.9834
0.9835
0.9837
0.9836
0.9835
0.9834
0.9838
0.9836
0.9836
0.9829
0.9828
0.983
0.9832
0.9835
0.9832
0.9834
0.9836
0.9837
0.9835
0.9837
0.9839
0.9842
0.9845
0.9845
0.9844
0.9844
0.9844
0.9844
0.9844
0.9844
0.9847
0.9846
0.9842
0.9836
0.9837
0.9838
0.9836
0.9834
0.9832
0.9835
0.984
0.984
0.9838
0.9837
0.9842
0.9847
0.9845
0.9845
0.9846
0.9846
0.9844
0.9845
0.9843
0.9844
0.9845
0.9845
0.9842
0.9845
0.9847
0.985
0.9852
0.9853
0.9855
0.985
0.9849
0.9847
0.9847
0.9845
0.984
0.9839
0.9836
0.9838
0.9838
0.9838
0.9839
0.9841
0.9845
0.9846
0.9848
0.9849
0.9847
0.9846
0.9844
0.9843
0.984
0.9842
0.9843
0.9839
0.984
0.9842
0.9847
0.9846
0.9849
0.9856
0.9855
0.9852
0.9852
0.9849
0.9852
0.9851
0.9849
0.9854
0.9854
0.985
0.9851
0.9851
0.9853
0.9852
0.9852
0.9852
0.9851
0.9853
0.9856
0.9856
0.9855
0.9855
0.9852
0.9855
0.9853
0.

0.9876
0.9875
0.9874
0.9877
0.9876
0.9873
0.9872
0.9869
0.9865
0.9865
0.9867
0.9866
0.9864
0.9865
0.9864
0.9861
0.9856
0.9854
0.9859
0.9859
0.9861
0.9861
0.9865
0.9865
0.9865
0.9865
0.9866
0.9868
0.9865
0.9865
0.9865
0.9863
0.9863
0.9864
0.9865
0.9863
0.9866
0.9863
0.9862
0.9862
0.9862
0.9858
0.9856
0.9858
0.9858
0.9857
0.9857
0.986
0.9859
0.9858
0.9862
0.9863
0.9862
0.9866
0.9869
0.9868
0.9867
0.9866
0.9863
0.9866
0.9868
0.987
0.9873
0.9874
0.9872
0.9875
0.9874
0.9871
0.9866
0.9862
0.9864
0.9862
0.9857
0.9849
0.9846
0.9845
0.9838
0.9832
0.9828
0.9823
0.982
0.9816
0.9813
0.9808
0.9809
0.9805
0.9802
0.9799
0.9796
0.9804
0.9804
0.9813
0.9825
0.9836
0.9847
0.9856
0.986
0.9863
0.9868
0.9871
0.9873
0.9876
0.9872
0.9873
0.987
0.9866
0.9863
0.9864
0.9863
0.9862
0.9862
0.9858
0.9858
0.9857
0.9858
0.9857
0.9858
0.9857
0.9857
0.9861
0.9861
0.9862
0.9863
0.9864
0.9868
0.9865
0.987
0.9871
0.9875
0.9873
0.9874
0.9875
0.9872
0.9872
0.9872
0.9873
0.9872
0.9873
0.9871
0.9869
0.9869
0.9867
0.9871
0.987

0.9879
0.9879
0.988
0.9881
0.9881
0.9879
0.9878
0.988
0.9879
0.9877
0.9873
0.9873
0.9874
0.9879
0.9881
0.9887
0.9888
0.9889
0.9888
0.9886
0.9883
0.9883
0.9884
0.9881
0.9879
0.9874
0.9872
0.9873
0.9875
0.9877
0.9874
0.9874
0.9882
0.9882
0.9882
0.9877
0.9875
0.9876
0.9876
0.9875
0.9872
0.987
0.9865
0.9866
0.9865
0.9862
0.9859
0.9859
0.9858
0.9856
0.9857
0.9857
0.9856
0.9857
0.9857
0.9861
0.9858
0.9859
0.986
0.9863
0.9866
0.9869
0.9872
0.9876
0.9878
0.9884
0.9886
0.9887
0.9887
0.9888
0.9886
0.9882
0.9883
0.9881
0.9878
0.9877
0.9872
0.9865
0.9868
0.9864
0.9863
0.9861
0.9858
0.9856
0.9856
0.9851
0.9851
0.985
0.985
0.9852
0.9853
0.9854
0.9854
0.9854
0.9852
0.9854
0.9854
0.9855
0.9858
0.9855
0.9857
0.9858
0.9857
0.9858
0.9857
0.9853
0.9852
0.9858
0.9859
0.9858
0.9864
0.9866
0.9867
0.9866
0.9868
0.9867
0.9869
0.9869
0.987
0.9871
0.9874
0.9879
0.9883
0.9882
0.9879
0.9876
0.9876
0.9872
0.987
0.9869
0.9866
0.9866
0.9869
0.9868
0.9867
0.9865
0.9864
0.9868
0.9866
0.9867
0.9867
0.9867
0.9867
0.9867


0.9883
0.9882
0.9882
0.9883
0.9883
0.9883
0.9883
0.9884
0.9881
0.9884
0.9888
0.9891
0.989
0.9891
0.9891
0.9891
0.989
0.9891
0.9891
0.9888
0.9887
0.9883
0.9884
0.9882
0.988
0.988
0.988
0.9878
0.9878
0.988
0.988
0.988
0.9882
0.9881
0.9885
0.9888
0.9887
0.9887
0.9886
0.9886
0.9885
0.9886
0.9884
0.9884
0.9884
0.9884
0.9881
0.9876
0.9878
0.988
0.9884
0.9886
0.9886
0.9888
0.989
0.9888
0.9888
0.9888
0.9885
0.9885
0.9885
0.9887
0.9888
0.9885
0.9886
0.9886
0.9886
0.9886
0.9883
0.9883
0.9885
0.9883
0.9884
0.9883
0.988
0.9881
0.9881
0.988
0.9881
0.988
0.988
0.9879
0.9878
0.9876
0.9876
0.9874
0.9873
0.9872
0.9869
0.9866
0.9863
0.986
0.9859
0.9859
0.9856
0.9856
0.9855
0.9855
0.9855
0.986
0.9863
0.9867
0.9869
0.9872
0.9873
0.9876
0.9876
0.9875
0.9875
0.9877
0.9876
0.9879
0.9878
0.9877
0.9878
0.988
0.988
0.988
0.9884
0.9885
0.9886
0.9884
0.9885
0.9883
0.9882
0.9881
0.9881
0.9881
0.9879
0.988
0.9876
0.9877
0.9873
0.9874
0.9868
0.9871
0.9869
0.987
0.987
0.9872
0.9871
0.9869
0.9869
0.9869
0.9872
0.988
0

0.9871
0.9871
0.9872
0.9872
0.9874
0.9876
0.9878
0.9878
0.9879
0.9877
0.9881
0.9884
0.9884
0.9888
0.9889
0.989
0.9889
0.989
0.9892
0.9893
0.9894
0.9895
0.9893
0.9896
0.9895
0.9895
0.9893
0.9894
0.9896
0.9895
0.9896
0.9897
0.9895
0.9892
0.989
0.989
0.9891
0.989
0.9891
0.9891
0.9892
0.9891
0.9893
0.9893
0.9891
0.9886
0.9887
0.9887
0.9884
0.9882
0.9879
0.9879
0.9882
0.9887
0.9891
0.9893
0.9892
0.9893
0.9894
0.9894
0.9894
0.9892
0.9893
0.9891
0.9891
0.9892
0.9891
0.9891
0.9892
0.9894
0.9893
0.9894
0.989
0.9891
0.9891
0.9894
0.9894
0.9894
0.9894
0.9893
0.9889
0.989
0.9892
0.9892
0.9897
0.9897
0.9899
0.9897
0.9896
0.9895
0.9894
0.9895
0.9898
0.9898
0.9897
0.9894
0.9892
0.9889
0.9888
0.989
0.9886
0.9885
0.9885
0.9885
0.9884
0.9882
0.9884
0.9883
0.9883
0.9881
0.9878
0.9874
0.9872
0.9872
0.9871
0.9871
0.987
0.987
0.9871
0.9872
0.9871
0.9873
0.9873
0.9875
0.9875
0.988
0.9887
0.9888
0.989
0.9891
0.9894
0.9895
0.9895
0.9896
0.9896
0.9895
0.9892
0.989
0.989
0.9891
0.9891
0.9891
0.9891
0.9893
0.9894

0.9906
0.9908
0.991
0.9908
0.9908
0.9908
0.9907
0.9908
0.991
0.991
0.9909
0.9908
0.9911
0.9912
0.991
0.9909
0.9908
0.9906
0.9908
0.9908
0.9907
0.9907
0.9906
0.9907
0.991
0.9907
0.9903
0.99
0.9898
0.9897
0.9898
0.9898
0.9897
0.9897
0.9896
0.9898
0.9898
0.9895
0.9895
0.9895
0.9894
0.9894
0.9894
0.9894
0.9893
0.9893
0.9893
0.9894
0.9891
0.9893
0.9895
0.9897
0.9897
0.9898
0.9898
0.99
0.9899
0.9897
0.9899
0.99
0.99
0.9899
0.9898
0.9898
0.9898
0.9898
0.9897
0.9897
0.9897
0.9894
0.9892
0.9892
0.9896
0.9894
0.9896
0.9893
0.9895
0.9893
0.9891
0.989
0.9892
0.9893
0.9898
0.9899
0.99
0.9903
0.9903
0.9905
0.9906
0.9905
0.9901
0.9904
0.9904
0.9905
0.9906
0.9906
0.9908
0.9907
0.9909
0.991
0.9913
0.9915
0.9919
0.9912
0.991
0.9909
0.9909
0.9907
0.9907
0.9905
0.9901
0.99
0.99
0.99
0.9901
0.9903
0.9902
0.9899
0.9901
0.9899
0.9898
0.9899
0.9899
0.9898
0.9899
0.9901
0.99
0.9897
0.9899
0.9898
0.9897
0.9897
0.9902
0.9903
0.9903
0.9903
0.9905
0.9905
0.9906
0.9906
0.9905
0.9905
0.9905
0.9904
0.9905
0.9905
0.99